<a href="https://colab.research.google.com/github/chicago007/test2/blob/main/test241016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pykrx
!pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512608 sha256=c3ac8174c342ab9726f663d8e0a03c6b6abf8f036d6ea7a0dde76fbf79d2812b
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [ ]:
#2. simple fastdtw with OHLC
import pandas as pd
from pykrx import stock
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

def get_stock_data(ticker, start_date, end_date):
    df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
    return df

def normalize_data(df):
    result = df.copy()
    for column in ['시가', '고가', '저가', '종가']:
        if df[column].max() == 0 and df[column].min() == 0:
            result[column] = df[column]
        else:
            result[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    return result

def clean_data(df):
    return df.dropna()

def compute_dtw_distance(stock_data_1, stock_data_2):
    distance, _ = fastdtw(stock_data_1, stock_data_2, dist=euclidean)
    return distance

def process_ticker(ticker, target_series, start_date, end_date):
    stock_data = get_stock_data(ticker, start_date, end_date)
    stock_data = clean_data(stock_data)
    normalized_data = normalize_data(stock_data[['시가', '고가', '저가', '종가']])

    if len(normalized_data) < min_data:
        return ticker, float('inf')

    comparison_series = normalized_data.values
    distance = compute_dtw_distance(target_series, comparison_series)
    return ticker, distance

def find_similar_stocks_sequential(target_data, start_date, end_date, tickers, top_n=20):
    target_series = target_data[['시가', '고가', '저가', '종가']].values
    print(target_series)
    distances = {}
    for ticker in tickers:
        print(ticker)
        ticker, distance = process_ticker(ticker, target_series, start_date, end_date)
        distances[ticker] = distance

    sorted_tickers = sorted(distances, key=distances.get)[:top_n]
    return sorted_tickers

def read_tickers():
    df = pd.read_csv('tickers0.csv')
    df['ticker'] = df['ticker'].str.strip("'")
    return df['ticker'].tolist()

# 메인 실행 부분
tickers = read_tickers()

ticker = '003160'
start_date = '20240327'
end_date = '20240618'
target_stock_data = get_stock_data(ticker, start_date, end_date)
target_stock_data = clean_data(target_stock_data)
target_stock_data = normalize_data(target_stock_data)
min_data = len(target_stock_data)

similar_stocks = find_similar_stocks_sequential(target_stock_data, '20240404', "20240627", tickers)
print("유사한 패턴을 가진 종목:", similar_stocks)

[[0.         0.         0.         0.        ]
 [0.05395418 0.06995336 0.07295374 0.05531295]
 [0.11899483 0.2158561  0.15747331 0.15938865]
 [0.25351072 0.16389074 0.22686833 0.12008734]
 [0.19438285 0.1152565  0.15925267 0.08369723]
 [0.12121212 0.19586942 0.113879   0.22852984]
 [0.27568367 0.42704863 0.32829181 0.40611354]
 [0.43828529 0.39906729 0.51067616 0.35225619]
 [0.41389505 0.31379081 0.41370107 0.32605531]
 [0.37620103 0.37508328 0.40124555 0.37409025]
 [0.40502587 0.51165889 0.48843416 0.51091703]
 [0.58610495 0.62025316 0.70996441 0.60989811]
 [0.67479675 0.82345103 0.80871886 0.87991266]
 [0.90391722 0.75349767 0.79359431 0.61935953]
 [0.70953437 0.70686209 0.85142349 0.70524017]
 [0.77087953 0.97001999 0.88078292 1.        ]
 [0.98891353 0.84676882 0.94217082 0.76710335]
 [0.72875092 0.61292472 0.75088968 0.61644833]
 [0.69992609 0.57561626 0.75266904 0.60189229]
 [0.71544715 0.6302465  0.84074733 0.63682678]
 [0.66518847 0.60559627 0.76423488 0.60917031]
 [0.72875092 

KeyboardInterrupt: 